In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
%matplotlib inline  # Ensures plots display in the notebook

In [ ]:
df = pd.read_csv('data/benin.csv')

In [ ]:
# Summary statistics
summary_stats = df.describe()
print(summary_stats)

# Missing values
missing_values = df.isna().sum()
print("\nMissing Values:\n", missing_values)

# Columns with >5% missing
missing_percentage = (missing_values / len(df)) * 100
high_missing = missing_percentage[missing_percentage > 5]
print("\nColumns with >5% missing values:\n", high_missing)